In [1]:
import numpy as np
from sklearn.model_selection import ParameterSampler
import json
from sklearn.model_selection import train_test_split
import pandas as pd

In [2]:
df=pd.read_hdf('/scratch/snx3000/nchernya/bregression/output_root/applied_res_2018-03-29_ZHbbll_morevar.hd5',columns=None)

In [3]:
training = '2018-03-27_job23_rawJetsJECtarget_us_plus_pisa_final2016'

In [4]:
previous_event = df['event'][0]
pts = [0,0,0,0,0,0,0]
etas = [0,0,0,0,0,0,0]
phis = [0,0,0,0,0,0,0]
masss = [0,0,0,0,0,0,0]
corrs = [0,0,0,0,0,0,0]
ress = [0,0,0,0,0,0,0]
jecs = [0,0,0,0,0,0,0]
jers = [0,0,0,0,0,0,0]
pts[0] = df['Jet_pt_raw'][0]*df['Jet_corr_JEC'][0]
corrs[0] =   df['Jet_pt_reg_NN_%s'%training][0]
ress[0] =   df['Jet_resolution_NN_%s'%training][0]
etas[0] = df['Jet_phi'][0]
phis[0] = df['Jet_eta'][0]
masss[0] = df['Jet_mass'][0]*df['Jet_corr_JEC'][0] #mass is already raw
jecs = df['Jet_corr_JEC'][0]
jers = df['Jet_corr_JER'][0]
final_pts = []
final_etas = []
final_masss = []
final_phis = []
final_corrs = []
final_ress = []
final_jecs = []
final_jers = []
max_idx = []


idx =0
for index in range(1,df.shape[0]-1):
#for index in range(1,349144):
    if ( df['event'][index]==previous_event ): 
        idx+=1
        pts[idx] = df['Jet_pt_raw'][index]*df['Jet_corr_JEC'][index]
        etas[idx] = df['Jet_eta'][index]
        phis[idx] = df['Jet_phi'][index] 
        masss[idx] = df['Jet_mass'][index]*df['Jet_corr_JEC'][index]
        corrs[idx] = df['Jet_pt_reg_NN_%s'%training][index]
        ress[idx] = df['Jet_resolution_NN_%s'%training][index]
        jecs[idx] =   df['Jet_corr_JEC'][index]
        jers[idx] =   df['Jet_corr_JER'][index]

    else : 
        final_pts.append(pts)
        final_etas.append(etas)
        final_phis.append(phis)
        final_masss.append(masss)
        final_corrs.append(corrs)
        final_ress.append(ress)
        final_jecs.append(jecs)
        final_jers.append(jers)
        max_idx.append(idx)
        pts = [0,0,0,0,0,0,0]
        etas = [0,0,0,0,0,0,0]
        phis = [0,0,0,0,0,0,0]
        masss = [0,0,0,0,0,0,0]
        corrs = [0,0,0,0,0,0,0]
        ress = [0,0,0,0,0,0,0]
        jecs = [0,0,0,0,0,0,0]
        jers = [0,0,0,0,0,0,0]
        idx=0
        pts[idx] = df['Jet_pt_raw'][index]*df['Jet_corr_JEC'][index]
        etas[idx] = df['Jet_eta'][index]
        phis[idx] = df['Jet_phi'][index] 
        masss[idx] = df['Jet_mass'][index]*df['Jet_corr_JEC'][index]
        corrs[idx] = df['Jet_pt_reg_NN_%s'%training][index]
        ress[idx] = df['Jet_resolution_NN_%s'%training][index]
        jecs[idx] =   df['Jet_corr_JEC'][index]
        jers[idx] =   df['Jet_corr_JER'][index]
        previous_event = df['event'][index]
        
        

In [ ]:
print(np.array(max_idx).max())
a = np.where( np.array(max_idx)> 2)
print(len(np.array(max_idx)[a]))

In [ ]:
counter=0
for i in range(0,len(final_pts)):
    if final_pts[i][0]!=0 and final_pts[i][1]!=0 : counter+=1

In [ ]:
print(counter)

In [5]:
import ROOT
from ROOT import TFile, TH1F, TCanvas, gROOT, TLorentzVector,TH1,TH1F

Welcome to JupyROOT 6.10/02


In [6]:
data = pd.DataFrame({'Jet_pt': final_pts, 'Jet_mass': final_masss, 'Jet_phi' :final_phis,
                    'Jet_eta' : final_etas, 'Jet_breg_NN_corr':final_corrs,
                    'Jet_breg_NN_res':final_ress })


In [7]:
final_dijetM = []
final_dijetM_reg = []
final_dijetPt = []
final_dijetPt_reg = []
for index, row in data.iterrows():
    dijetM = 0
    dijetM_reg = 0
    dijetPt = 0
    dijetPt_reg = 0
    if (row['Jet_pt'][0]>20 and row['Jet_pt'][1]>20 
        and row['Jet_breg_NN_corr'][0] >0.5 and row['Jet_breg_NN_corr'][0] <2
       and row['Jet_breg_NN_corr'][1] >0.5 and row['Jet_breg_NN_corr'][1] <2)  : 
        jet1, jet2 = TLorentzVector(),TLorentzVector()
        jet1_reg, jet2_reg = TLorentzVector(),TLorentzVector()
        jet1.SetPtEtaPhiM(row['Jet_pt'][0],row['Jet_eta'][0],row['Jet_phi'][0],row['Jet_mass'][0])
        jet2.SetPtEtaPhiM(row['Jet_pt'][1],row['Jet_eta'][1],row['Jet_phi'][1],row['Jet_mass'][1])
        jet1_reg.SetPtEtaPhiM(row['Jet_pt'][0]*row['Jet_breg_NN_corr'][0],row['Jet_eta'][0],row['Jet_phi'][0],row['Jet_mass'][0])
        jet2_reg.SetPtEtaPhiM(row['Jet_pt'][1]*row['Jet_breg_NN_corr'][1],row['Jet_eta'][1],row['Jet_phi'][1],row['Jet_mass'][1])
        dijetM = (jet1+jet2).M()
        dijetM_reg = (jet1_reg+jet2_reg).M()
        dijetPt = (jet1+jet2).Pt()
        dijetPt_reg = (jet1_reg+jet2_reg).Pt()
    final_dijetM.append(dijetM)
    final_dijetM_reg.append(dijetM_reg)
    final_dijetPt.append(dijetPt)
    final_dijetPt_reg.append(dijetPt_reg)
        

In [8]:
data['dijetM_reg'] = final_dijetM_reg
data['dijetM'] = final_dijetM
data['dijetPt_reg'] = final_dijetPt_reg
data['dijetPt'] = final_dijetPt


In [9]:
out_dir = '/scratch/snx3000/nchernya/bregression/output_root/'
filename = 'ZHbll_dijetmass_pt_per_event'
data.to_hdf(out_dir+filename+'.hd5'  ,'w')

/users/musella/my-env/lib/python3.5/site-packages/pandas/core/generic.py:1282: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->['Jet_breg_NN_corr', 'Jet_breg_NN_res', 'Jet_eta', 'Jet_mass', 'Jet_phi', 'Jet_pt']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


In [ ]:
bins = np.linspace(50, 200, 100)
n,bins,patches = plt.hist(data[data.dijetM > 0]['dijetM'],bins=bins,color='blue',alpha=0.5)
n,bins,patches = plt.hist(data[data.dijetM_reg > 0]['dijetM_reg'],bins=bins,color='red',alpha=0.5)

In [ ]:
import sys
sys.path.append("/users/nchernya/HHbbgg_ETH/bregression/python/")
import plotting_utils as plotting

In [ ]:
#plotting.fit_dijetmass(data[data.dijetM_reg > 0]['dijetM_reg'],data[data.dijetM > 0]['dijetM'])
#plotting.plot_rel_pt_diff(data[data.dijetM_reg > 0]['dijetM_reg'],data[data.dijetM > 0]['dijetM'])
gROOT.SetBatch(True)
gROOT.ProcessLineSync(".x ~/HHbbgg_ETH/scripts/setTDRStyle.C")
gROOT.ForceStyle()
gStyle.SetPadTopMargin(0.06)
gStyle.SetPadRightMargin(0.04)
gStyle.SetPadLeftMargin(0.15)

In [ ]:
def fit_dijetmass(rel_diff_regressed,rel_diff,style=False,n_bins=100,outString=None):
    if style==True:
        gROOT.SetBatch(True)
        gROOT.ProcessLineSync(".x ~/HHbbgg_ETH/scripts/setTDRStyle.C")
        gROOT.ForceStyle()
        gStyle.SetPadTopMargin(0.06)
        gStyle.SetPadRightMargin(0.04)
        gStyle.SetPadLeftMargin(0.15)

    c_min = min(min(rel_diff_regressed),min(rel_diff))
    c_max = max(max(rel_diff_regressed),max(rel_diff))
    c_min=50.
    c_max=200.

    Histo_rel_diff = np.histogram(rel_diff,bins=n_bins,range=(c_min,c_max))
    Histo_rel_diff_reg = np.histogram(rel_diff_regressed,bins=n_bins,range=(c_min,c_max))

    h_rel_diff = TH1F("hrel_diff", "hrel_diff", n_bins, c_min, c_max)
    for i in range(len(rel_diff)):
        h_rel_diff.Fill(rel_diff[i])
    h_rel_diff_reg = TH1F("hrel_diff_reg", "hrel_diff_reg", n_bins, c_min, c_max)
    for i in range(len(rel_diff_regressed)):
        h_rel_diff_reg.Fill(rel_diff_regressed[i])
    h_rel_diff.SetLineColor(ROOT.kBlue)
    h_rel_diff.SetMarkerColor(ROOT.kBlue)
    h_rel_diff.SetLineWidth(2)
    h_rel_diff_reg.SetLineColor(ROOT.kRed)
    h_rel_diff_reg.SetMarkerColor(ROOT.kRed)
    h_rel_diff_reg.SetLineWidth(2)



    AllHistos= [Histo_rel_diff, Histo_rel_diff_reg]
    h_max = max([histo[0].max() for histo in AllHistos])*1.2
    h_min = min([histo[0].min() for histo in AllHistos])

    c = ROOT.TCanvas("c","c",900,900)
    c.cd()
    frame = TH1F("hframe", "hframe", n_bins, c_min, c_max)
    frame.SetStats(0)
    frame.GetXaxis().SetTitleOffset(0.91);
    frame.GetYaxis().SetTitle("Events")
  #  frame.GetXaxis().SetTitle("(p_{T}^{Reco}-p_{T}^{gen})/p_{T}^{gen}")
    frame.GetXaxis().SetTitle("p_{T}^{gen}/p_{T}^{reco}")
    frame.GetYaxis().SetLabelSize(0.04)
    frame.GetYaxis().SetRangeUser(h_min,h_max)

    frame.Draw()
    h_rel_diff.Draw("samePE")
    h_rel_diff_reg.Draw("samePE")
    print('Nominal : mean, RMS :',h_rel_diff.GetMean(), h_rel_diff.GetRMS())
    print('Regresesd : mean, RMS : ',h_rel_diff_reg.GetMean(), h_rel_diff_reg.GetRMS())






    leg = ROOT.TLegend(0.55,0.75,0.9,0.9)
    leg.SetFillStyle(-1)
    leg.SetBorderSize(0)
    leg.SetTextFont(42)
    leg.SetTextSize(0.03)
  #  leg.Draw()


    h_names = ['nom','reg']
    datahists = [h_rel_diff,h_rel_diff_reg]
    x=[]
    datahist=[]
    m=[]
    m_initial=[120.0103e+00,9.5381e-01]
    s=[]
    s_initial=[ 13.210,13.967]
    a=[]
    a_initial=[-7.7802e-01,-1.1260e+00]
    n=[]
    n_initial=[ 6.0149e+00,5.5622e+00]

    Ap,Xp,sigp,xi,rho1,rho2 = [],[],[],[],[],[]
    Xp_initial,sigp_initial,xi_initial,rho1_initial,rho2_initial =  120., 20.11,2.2695e-01, 6.4189e-02,  9.0282e-02
    meanr, sigmaL, sigmaR, alphaL, alphaR = [],[],[],[],[]

    fsig=[]
    sig=[]
    model=[]
    res=[]
    integral=[]
    formula=[]
    scale_factors=[]
    scaled_cb=[]
    func=[]
    colors=[ROOT.kBlue,ROOT.kRed]
    chi_squares=[]
    fwhm_bukin=[]
    fwhm=[]
    fwhm.append( -1 *(h_rel_diff.GetBinCenter( h_rel_diff.FindFirstBinAbove(h_rel_diff.GetMaximum()/2.)  )  - h_rel_diff.GetBinCenter( h_rel_diff.FindLastBinAbove(h_rel_diff.GetMaximum()/2.) ) )  )
    fwhm.append( -1 *(h_rel_diff_reg.GetBinCenter( h_rel_diff_reg.FindFirstBinAbove(h_rel_diff_reg.GetMaximum()/2.)  )  - h_rel_diff_reg.GetBinCenter( h_rel_diff_reg.FindLastBinAbove(h_rel_diff_reg.GetMaximum()/2.) ) )  )

    for num,h in enumerate(h_names):
        x.append(RooRealVar("x_%s"%h,"x_%s"%h,c_min,c_max))
        datahist.append(RooDataHist("roohist_%s"%h,"roohist_%s"%h,RooArgList(x[num]),datahists[num]))

      #######################Bukin function ##################

        Xp.append(RooRealVar("Xp_%s"%h,"Xp_%s"%h,Xp_initial,100,140.))
        sigp.append(RooRealVar("sigp_%s"%h,"sigp_%s"%h,sigp_initial,10,40))
        xi.append(RooRealVar("xi_%s"%h,"xi_%s"%h,xi_initial,-1,1))
        rho1.append(RooRealVar("rho1_%s"%h,"rho1_%s"%h,rho1_initial,-1,1)) #left
        rho2.append(RooRealVar("rho2_%s"%h,"rho2_%s"%h,rho2_initial,-1,1)) #right
        sig.append(RooBukinPdf("signal_bukin_%s"%h,"signal_bukin_%s"%h,x[num],Xp[num],sigp[num],xi[num],rho1[num],rho2[num]))
        res.append(sig[num].fitTo(datahist[num],ROOT.RooFit.Save(ROOT.kTRUE)))
        res[num].Print()

       # chi_squares.append((x[num].frame()).chiSquare())
        x[num].setRange("integralRange%s"%h, c_min,c_max)
        integral.append(sig[num].createIntegral(RooArgSet(x[num]), ROOT.RooFit.Range("integralRange%s"%h)))

        scale_factors.append(datahists[num].Integral()*datahists[num].GetBinWidth(1)/integral[num].getVal())
        scale_factors.append(datahists[num].Integral()*datahists[num].GetBinWidth(1)/integral[num].getVal())
      #  formula.append("%f *signal_gauss_%s"%(scale_factors[num],h))
        formula.append("%f *signal_bukin_%s"%(scale_factors[num],h))
     #   formula.append("%f *signal_cruijff_%s"%(scale_factors[num],h))
       # create a scaled  function = scale * function
        scaled_cb.append(RooFormulaVar("scaled_cb_%s"%h,formula[num],RooArgList(sig[num])))
        func.append(scaled_cb[num].asTF(RooArgList(x[num])))
        func[num].SetLineColor(colors[num])
        datahists[num].SetMarkerColor(colors[num])
        fwhm_bukin.append(sigp[num].getVal()*2*math.sqrt(2*math.log(2)))

      #  chi_squares.append(RooChi2Var("chi2_%s"%h,"chi2_%s"%h,sig[num],datahist[num]))


    fitfunc='Bukin'
  #  fitfunc='Bifurgaus'
    fit_result_file = std.ofstream(utils.IO.plotFolder+"../fitResults/fitResult_%s"%(fitfunc)+str(outString)+'.txt')
    res[0].floatParsFinal().printMultiline(fit_result_file, 1111, True)
    res[1].floatParsFinal().printMultiline(fit_result_file, 1111, True)
    fit_result_file.close()


    leg.AddEntry(h_rel_diff,"Nominal JEC" ,"P")
    leg.AddEntry(h_rel_diff_reg,"Regressed" ,"P")


    c2 = ROOT.TCanvas("c2","c2",900,900)
    c2.cd()
    frame.Draw()
    func[0].Draw("same")
    func[1].Draw("same")
    h_rel_diff.Draw("PEHISTsame")
    h_rel_diff_reg.Draw("PEHISTsame")
    leg.Draw()

    c2.SaveAs(utils.IO.plotFolder+"dijet_%s_"%(fitfunc)+str(outString)+'.png')
    c2.SaveAs(utils.IO.plotFolder+"dijet_%s_"%(fitfunc)+str(outString)+'.pdf')
    c2.Draw()


In [ ]:
import ROOT
import math
from array import array
from ROOT import std
from ROOT import gROOT
from ROOT import gStyle
from ROOT import TH1F, TGraph, TF1
from ROOT import TCanvas
from ROOT import RooRealVar, RooDataHist, RooFormulaVar, RooVoigtian, RooChebychev, RooArgList, \
                 RooArgSet, RooAddPdf, RooDataSet, RooCategory, RooSimultaneous, \
                 RooBreitWigner, RooCBShape, RooFFTConvPdf, RooBukinPdf, RooBifurGauss, RooGenericPdf, RooChi2Var


In [ ]:
fit_dijetmass(data[data.dijetM_reg > 0]['dijetM_reg'].values.reshape(-1,1),data[data.dijetM > 0]['dijetM'].values.reshape(-1,1))
